In [74]:
from glob import glob
from src.systems.humanoid.system import HumanoidSystem
from src.flow_matching.humanoid.latent_conditional.flow_matcher import HumanoidLatentConditionalFlowMatcher
from matplotlib import pyplot as plt

import numpy as np
import os
import torch

data_dir = "/common/users/shared/pracsys/genMoPlan/data_trajectories/humanoid_get_up_slow_reach"
roa_file = "/common/users/shared/pracsys/genMoPlan/data_trajectories/humanoid_get_up_slow_reach/roa_labels.txt"
bounds_file = "/common/users/dm1487/arcmg_datasets/humanoid_get_up/humanoid_data_bounds.pkl"
system = HumanoidSystem(bounds_file=bounds_file, use_dynamic_bounds=True, head_height_threshold=1.4, torso_z_threshold=0.9, speed_threshold=0.2)

ckpt_path = "/common/home/dm1487/robotics_research/tripods/olympics-classifier/outputs/humanoid_slow_reach/2025-11-02_23-19-45"
flow_matcher = HumanoidLatentConditionalFlowMatcher.load_from_checkpoint(ckpt_path, device="cuda:0")


✅ Loaded Humanoid bounds from: /common/users/dm1487/arcmg_datasets/humanoid_get_up/humanoid_data_bounds.pkl
📊 Per-Dimension Normalization Limits:

   Euclidean Block 1 (dims 0-33):
     [ 0]: ±  0.859
     [ 1]: ±  1.375
     [ 2]: ±  0.725
     [ 3]: ±  0.526
     [ 4]: ±  0.633
     ... (29 more dimensions)

   Sphere (dims 34-36): NO NORMALIZATION (unit norm)

   Euclidean Block 2 (dims 37-66):
     [37]: ±  1.914
     [38]: ±  1.916
     [39]: ±  3.328
     [40]: ±  2.790
     [41]: ±  2.940
     ... (25 more dimensions)

Loaded Humanoid bounds from: /common/users/dm1487/arcmg_datasets/humanoid_get_up/humanoid_data_bounds.pkl
📁 Folder provided: /common/home/dm1487/robotics_research/tripods/olympics-classifier/outputs/humanoid_slow_reach/2025-11-02_23-19-45
🔍 Searching for checkpoint in folder...
   ✓ Found best checkpoint (val_loss=0.0597)
   📄 Using: epoch110-val_loss0.0597.ckpt
🤖 Loading Humanoid LCFM checkpoint: /common/home/dm1487/robotics_research/tripods/olympics-classifier/o

In [75]:
roa_data = np.loadtxt(roa_file, delimiter=",")
inp, labels = roa_data[:, :67], roa_data[:, 67]
inp = torch.from_numpy(inp).float().to("cuda:0")
labels = torch.from_numpy(labels).long().to("cuda:0")
np.mean(roa_data[:, 67] == 1)


0.4597333333333333

In [77]:
# from tqdm import tqdm
# samples = 20
# repeats = 1

# tp = 0
# tn = 0
# fp = 0
# fn = 0

# # idx = 2049
# for idx in tqdm(range(5, len(roa_data))):
#     all_head_heights = []
#     for _ in range(samples):
#         model_input = inp[idx:idx+1, :].clone()
#         head_height = []
#         for i in range(repeats):
#             pred = flow_matcher.predict_endpoint(model_input)
#             head_height.append(pred[:, 21].item())
#             model_input = pred.clone()
#         all_head_heights.append(head_height)
#     is_success = (sum(np.array(all_head_heights) > 1.3)/samples) > 0.5
#     # print(sum(np.array(all_head_heights) > 1.3)/samples, is_success, labels[idx])
#     if labels[idx] == 1 and is_success:
#         tp += 1
#     elif labels[idx] == 0 and is_success:
#         fp += 1
#     elif labels[idx] == 1 and not is_success:
#         fn += 1
#     elif labels[idx] == 0 and not is_success:
#         tn += 1
        
#     if idx == 50:
#         break

In [85]:
from tqdm import tqdm

samples = 100
repeats = 1
batch_size = 2048  # You can tune this depending on memory
success_threshold = 0.55
failure_threshold = 0.45
tp = 0
tn = 0
fp = 0
fn = 0

sep_count = 0

start_idx = 5
stop_idx = len(roa_data)

for batch_start in tqdm(range(start_idx, stop_idx, batch_size)):
    batch_end = min(batch_start + batch_size, stop_idx)
    batch_inp = inp[batch_start:batch_end, :]

    # Will be (batch_size, samples, repeats)
    all_head_heights = np.zeros((batch_end - batch_start, samples, repeats))

    for sample_idx in range(samples):
        model_input = batch_inp.clone()
        for repeat_idx in range(repeats):
            pred = flow_matcher.predict_endpoint(model_input)
            # pred shape: (batch_size, d)
            all_head_heights[:, sample_idx, repeat_idx] = pred[:, 21].detach().cpu().numpy()
            model_input = pred.clone()

    # Success logic: check along [samples, repeats] for each data point in batch
    # We'll take the mean across all samples and repeats for head height > 1.3
    mean_success = (all_head_heights > 1.3).mean(axis=(1,2))
    is_success = mean_success > success_threshold  # shape: (batch_size,)
    
    is_failure = mean_success < failure_threshold  # shape: (batch_size,)
    
    sep_count += np.sum((mean_success <= success_threshold) & (mean_success >= failure_threshold))

    batch_labels = labels[batch_start:batch_end].cpu().numpy()  # shape: (batch_size,)

    tp += np.sum((batch_labels == 1) & (is_success))
    fp += np.sum((batch_labels == 0) & (is_success))
    fn += np.sum((batch_labels == 1) & (is_failure))
    tn += np.sum((batch_labels == 0) & (is_failure))

100%|██████████| 8/8 [00:55<00:00,  6.96s/it]


In [84]:
precision = tp / (tp + fp)
recall = tp / (tp + fn)
f1 = 2 * precision * recall / (precision + recall)
specificity = tn / (tn + fp)
sep_perc = sep_count/len(roa_data)
print(f"Precision: {precision}, Recall: {recall}, F1: {f1}, Specificity: {specificity}, Sep: {sep_perc}")

# confusion matrix
conf_mat = np.zeros((2, 2))
conf_mat[0, 0] = tp
conf_mat[0, 1] = fp
conf_mat[1, 0] = fn
conf_mat[1, 1] = tn


Precision: 0.480561122244489, Recall: 0.633051742344245, F1: 0.5463659147869674, Specificity: 0.39748953974895396, Sep: 0.73


In [70]:
conf_mat

array([[3040., 3342.],
       [1158., 1377.]])

In [145]:
success_count = 0
failure_count = 0
success_lengths = []
failure_lengths = []
from tqdm import tqdm

for file in tqdm(glob(os.path.join(data_dir, "trajectories/*.txt"))):
    data = np.loadtxt(file, delimiter=",")
    success = False
    if system.is_in_attractor(data[-1:]).item():
        success_count += 1
        success_lengths.append(data.shape[0])
    else:
        failure_count += 1
        failure_lengths.append(data.shape[0])
        
    if (success_count + failure_count) % 1000 == 0:
        print(f"Success count: {success_count}")
        print(f"Failure count: {failure_count}")
        print(f"Success rate: {success_count / (success_count + failure_count)}")
        print(f"Failure rate: {failure_count / (success_count + failure_count)}")
        print(f"Success lengths: {np.median(success_lengths)}")
        print(f"Failure lengths: {np.median(failure_lengths)}")

  7%|▋         | 1012/15000 [00:07<01:47, 130.12it/s]

Success count: 403
Failure count: 597
Success rate: 0.403
Failure rate: 0.597
Success lengths: 446.0
Failure lengths: 436.0


 13%|█▎        | 2012/15000 [00:15<01:39, 130.12it/s]

Success count: 784
Failure count: 1216
Success rate: 0.392
Failure rate: 0.608
Success lengths: 446.0
Failure lengths: 438.5


 20%|██        | 3017/15000 [00:23<01:33, 127.49it/s]

...

Success count: 5007
Failure count: 7993
Success rate: 0.3851538461538462
Failure rate: 0.6148461538461538
Success lengths: 441.0
Failure lengths: 436.0


 93%|█████████▎| 14023/15000 [01:48<00:07, 130.99it/s]

Success count: 5388
Failure count: 8612
Success rate: 0.38485714285714284
Failure rate: 0.6151428571428571
Success lengths: 441.0
Failure lengths: 436.0


100%|██████████| 15000/15000 [01:56<00:00, 128.74it/s]

Success count: 5747
Failure count: 9253
Success rate: 0.3831333333333333
Failure rate: 0.6168666666666667
Success lengths: 441.0
Failure lengths: 436.0


In [147]:

print(f"Success count: {success_count}")
print(f"Failure count: {failure_count}")
print(f"Success rate: {success_count / (success_count + failure_count)}")
print(f"Failure rate: {failure_count / (success_count + failure_count)}")

Success count: 5747
Failure count: 9253
Success rate: 0.3831333333333333
Failure rate: 0.6168666666666667


In [30]:
print(f"Success lengths: {np.median(success_lengths)}")
print(f"Failure lengths: {np.median(failure_lengths)}")

Success lengths: 192.0
Failure lengths: 441.0
